In [ ]:
import pandas as pd
import numpy as pn
import panel as pn
pn.extension('tabulator')
import matplotlib.pyplot as plt     
from matplotlib import rcParams
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh','matplotlib')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv(r"C:\Users\nizam\OneDrive\Documents\Portfolio\Python Dashboard\victims-of-selected-major-offences\victims-of-selected-major-offences-by-age-group.csv")
df.sample(10)

### Univariate Statistics

In [ ]:
output_df = pd.DataFrame(columns=['Count', 'Missing', 'Unique', 'Dtype', 'Numeric', 'Mode', 'Mean', 'Min', '25%', 'Median', '75%', 'Max', 'Std', 'Skew', 'Kurt'])

for col in df:
    if pd.api.types.is_numeric_dtype(df[col]):
        output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype, pd.api.types.is_numeric_dtype(df[col]), df[col].mode().values[0], df[col].mean(), df[col].min(), df[col].quantile(.25), df[col].median(), df[col].quantile(.75), df[col].max(), df[col].std(), df[col].skew(), df[col].kurt()]
    else:
        output_df.loc[col] = [df[col].count(), df[col].isnull().sum(), df[col].nunique(), df[col].dtype, pd.api.types.is_numeric_dtype(df[col]), df[col].mode().values[0], '-', '-', '-', '-', '-', '-', '-', '-', '-']

output_df.sort_values(by=['Numeric', 'Skew', 'Unique'], ascending=False)


#### Creating Gender Column and Changing Column Data Format

In [ ]:
df['Gender'] = df['level_1'].str.split().str.get(0)
df = df[df['Gender'] != 'Total']
df

In [ ]:
df['level_1'] = df['level_1'].str.split(n=3).str[3]
df

#### Rename Columns

In [ ]:
df = df.rename(columns={'level_1' : 'Type of Offence', 'level_2':'Victim Age Group', 'value':'No. of Victims','year':'Year'})
df

#### Creating interactive dashboard

In [ ]:
idf = df.interactive()

#### Creating lists for plots

In [ ]:
offence_list = list(df['Type of Offence'].unique())
age_group_list = list(df['Victim Age Group'].unique())
gender_list = list(df['Gender'].unique())
yearlist = list(df['Year'].unique())

### No. of Offences Reported per Year

In [ ]:
# Radio buttons for Gender

genderselect = pn.widgets.CheckButtonGroup(name='Gender', value = gender_list, options=gender_list, button_type='success')

genderselect

In [ ]:
# Radio buttons for Age Group

ageselect = pn.widgets.CheckBoxGroup(name='Victim Age Group', value = age_group_list, options=age_group_list)

ageselect

In [ ]:
# Pipeline for Line Plot for Total Offences Reported


totaloffence_pipeline = (idf[(idf['Gender'].isin(genderselect)) & idf['Victim Age Group'].isin(ageselect)].groupby(['Year'])['No. of Victims'].sum()
                    .to_frame().reset_index().sort_values(by='Year').reset_index(drop=True))

In [ ]:
# Line Plot for Total Offences Reported

totaloffenceplot = totaloffence_pipeline.hvplot(x='Year', y='No. of Victims', width=1100, height=400, color=hv.Cycle('Set1'))
totaloffenceplot

#### No. of Victims by Category

In [ ]:
# Buttons for Years

yearselect = pn.widgets.CheckButtonGroup(name='Year', value=yearlist, options=yearlist, button_type='success')

yearselect

In [ ]:
offencetypepipeline = (idf[(idf['Year'].isin(yearselect)) & idf['Victim Age Group'].isin(ageselect)]
                    .groupby(['Type of Offence','Gender'])['No. of Victims'].sum()
                    .to_frame().reset_index().sort_values(by='Type of Offence').reset_index(drop=True))

offencetypepipeline

In [ ]:
offenctypeeplot = offencetypepipeline.hvplot(x='Type of Offence', by='Gender', y='No. of Victims', kind='bar', 
                                            height=400, width=1150, rot=90, color=hv.Cycle('Set1'))

offenctypeeplot

In [ ]:
template = pn.template.FastListTemplate(
    title="Victims of Selected Major Selected Offences (2011 - 2020)",
    sidebar=[pn.pane.Markdown("## Victims of Selected Major Offences by Age Group"),
            pn.pane.Markdown('#### Figures are based on Annual Statistical Reports on Crime in Singapore'),
            pn.pane.Markdown('#### Select Age Group of Interest:'), ageselect,
            pn.pane.Markdown('[Created By: Nizam Sulaiman](https://portfolio.nurnizam.com)')],
    main=[pn.Row(pn.Column(genderselect,totaloffenceplot.panel(width=750)),sizing_mode='stretch_width'),
            pn.Row(pn.Column(pn.Row(yearselect),(offenctypeeplot.panel(width=750))),sizing_mode='stretch_width'),
            pn.Row("Source: [Data.gov.sg](https://data.gov.sg/dataset/victims-of-selected-major-selected-offences)")],
            accent_base_color='#88d8b0',
            header_background='#88d8b0',
            theme_toggle=False
)

template.show()


In [ ]:
# template.servable()